# Homework 6
任课老师：胡俊峰

负责助教：苏亚鲁、李浩然

注意：仅需要提交.ipynb文件，请**不要**将下发压缩包中的其他文件一并交上。

截止日期：3月18日24点

## 第零部分 代码理解
### 0.1 阅读代码预测结果，分析结果是否如预期，并按要求填充代码行，实现功能。

In [1]:
import pandas as pd
df = pd.DataFrame([[1, 2], [3, 4]], columns = ['a','b'])
print(df)
print(df['a']) 

# 预期
#   a b
# 0 1 2
# 1 3 4

# 0 1
# 1 3

   a  b
0  1  2
1  3  4
0    1
1    3
Name: a, dtype: int64


In [2]:
# 理解那个是0号索引
dft = df.T           
print(dft)
print(dft[1]['a'])  #  print(dft['a'][1] ?)

# 预测
#   0 1
# a 1 3
# b 2 4

# 3

# 用dft['a'][1]会报KeyError

   0  1
a  1  3
b  2  4
3


In [3]:
df2 = pd.DataFrame([[5, 6], [7, 8]], columns = ['a','b'])
df3 = pd.concat([df,df2])  # 尝试用一行代码，df1，df2的横向拼接？
print(df3)

# 预测
#   a b
# 0 1 2
# 1 3 4
# 0 5 6
# 1 7 8

   a  b
0  1  2
1  3  4
0  5  6
1  7  8


In [4]:
# 请填充一行代码，实现打印 ‘b’列的值 <6 的df3的子视图
print(df3['b'][df3['b'] < 6])

0    2
1    4
Name: b, dtype: int64


In [5]:
df = pd.DataFrame([[4, 9]] * 3, columns=['A', 'B'])
print(df)
# 预测
#   A B
# 0 4 9
# 1 4 9
# 2 4 9

   A  B
0  4  9
1  4  9
2  4  9


dataframe.apply(function,axis)对一行或一列做出一些操作（axis=1遍历行，axis=0遍历列），可以使用匿名函数。

In [6]:
import numpy as np
print(df.apply(np.sum))
print(df.apply(np.sum, axis=1))
# 预测
#  A  B
# 12 27

# 0 13
# 1 13
# 2 13

A    12
B    27
dtype: int64
0    13
1    13
2    13
dtype: int64


In [7]:
df.apply(lambda x: [1, 2], axis=1)
# 预测
# 0 [1, 2]
# 1 [1, 2]
# 2 [1, 2]

0    [1, 2]
1    [1, 2]
2    [1, 2]
dtype: object

In [8]:
df.apply(lambda x: [1, 2], axis=1, result_type='expand')
# 预测
#   0 1
# 0 1 2
# 1 1 2
# 2 1 2

,0,1
0,1,2
1,1,2
2,1,2


### 0.2 请认真阅读并运行以下示例代码，并思考C/S的多进程、多线程以及下面代码中select实现方式的优缺点。
注：文件夹下的example_client.py为本思考题对应的客户端文件。client.py为第一部分网络编程对应的客户端文件。（两个客户端均无需改动）

In [9]:
import select
from socket import *
import sys
import queue

# 生成socket对象
server = socket(AF_INET, SOCK_STREAM)
server.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
server.setblocking(False)

# 绑定地址，设置监听
server.bind(('localhost',9999))
server.listen(5)

# 将自己也放进待监测列表里
inputs = [server, ]
outputs = []
message_queues = {}

while True:
    rlist, wlist, elist = select.select(inputs,outputs,inputs) #如果没有任何fd就绪,那程序就会一直阻塞在这里

    for r in rlist:  # 遍历已经可以准备读取数据的 fd
        if r is server: # 如果这个 fd 是server，即 server 有数据待接收读取，说明有新的客户端连接过来了
            conn, client_addr = r.accept()
            print("new connection from",client_addr)
            conn.setblocking(False)
            inputs.append(conn) # 将这个新的客户端连接添加到检测的列表中
            message_queues[conn] = queue.Queue() # 用队列存储客户端发送来的数据，等待服务器统一返回数据

        else:          # 这个可读的 r 不是服务器，那就是某个客户端。就是说客户端发送数据过来了，这些数据处于待读取状态
            try:       # 异常处理，这是为了防止客户端异常断开报错（比如手动关掉客户端黑窗口，服务器也会跟着报错退出）
                data = r.recv(1024)
                if data:    # 根据判断data是否为空，判断客户端是否断开
                    print("收到来自[%s]的数据:" % r.getpeername()[0], data)
                    message_queues[r].put(data)   # 收到的数据先放到queue里,一会返回给客户端
                    if r not in outputs:
                        outputs.append(r)     # 放进可写的fd列表中，表明这些 fd 已经准备好去发送数据了。
                else:   # 如果数据为空，表明客户端断开了
                    print('客户端断开了')
                    if r in outputs:
                        outputs.remove(r)    #  清理已断开的连接
                    inputs.remove(r)         # 清理已断开的连接
                    del message_queues[r]    # 清理已断开的连接
            except ConnectionResetError:     # 如果报错，说明客户端断开了
                print("客户端异常断开了", r)
                if r in outputs:
                    outputs.remove(r)   # 清理已断开的连接
                inputs.remove(r)        # 清理已断开的连接
                del message_queues[r]  # 清理已断开的连接

    for w in wlist:       # 遍历可写的 fd 列表，即准备好发送数据的那些fd
        # 判断队列是否为空
        try :
            next_msg = message_queues[w].get_nowait()
        except queue.Empty:
            # print("client [%s]" % w.getpeername()[0], "queue is empty..")
            outputs.remove(w)
        # 队列不为空，就把队列中的数据改成大写，原样发回去
        else:
            # print("sending msg to [%s]"% w.getpeername()[0], next_msg)
            w.send(next_msg.upper())

    for e in elist:   # 处理报错的 fd
        e.close()
        print("Error occured in ",e.getpeername())
        inputs.remove(e)
        if e in outputs:
            outputs.remove(e)
        del message_queues[e]

new connection from ('127.0.0.1', 7916)
new connection from ('127.0.0.1', 7917)
收到来自[127.0.0.1]的数据: b'This is the message. '
收到来自[127.0.0.1]的数据: b'This is the message. '
收到来自[127.0.0.1]的数据: b'It will be sent '
收到来自[127.0.0.1]的数据: b'It will be sent '
收到来自[127.0.0.1]的数据: b'in parts.'
收到来自[127.0.0.1]的数据: b'in parts.'
客户端断开了
客户端断开了


KeyboardInterrupt: 

## 第一部分：网络编程 (2.5分)

### 请将下列代码补充完整，从而实现如下功能：
**功能：**
* 为每个socket注册用户名，并检查用户名是否已经存在。若已经存在，则发送'Username already exists!'；否则发送'Welcome!'。(1.5分)
* 将注册成功的用户发送的信息，转发给其他的注册成功的用户。(1分)

**注：**
当补充完整代码后，可以运行client.py进行检验，其预期的输出为：\
connecting to localhost port 服务器端口 \
Welcome!to 第一个客户端端口 \
Username already exists!to 第二个客户端端口 \
Welcome!to 第二个客户端端口 \
Hello everybody!to 第二个客户端端口 

In [146]:
from socket import *
from select import select
import queue

class Server:
    def __init__(self, host, port):
        self.server = socket(AF_INET, SOCK_STREAM)
        self.server.setsockopt(SOL_SOCKET, SO_REUSEADDR, 1)
        self.server.setblocking(False)

        self.server.bind((host, port))
        self.server.listen(5)
        self.rlist = [self.server]
        self.wlist = []
        self.message_queues = {}
        self.users = {}  # 用于判断socket是否已经注册用户名

    def accept(self):
        while True:
            rs, ws, xs = select(self.rlist, self.wlist, self.rlist)
            for r in rs:
                if r is self.server:
                    # 服务器接受客户端连接
                    conn, addr = self.server.accept()
                    print("new connection from",addr)
                    conn.setblocking(False)
                    self.rlist.append(conn)
                    self.message_queues[conn] = queue.Queue()
                    self.users[conn] = False
                else:
                    try:
                        # 服务器接受客户端的消息并转发给所有客户端
                        data = r.recv(1024)
                        if data:
                            print("收到来自[%s]的数据:" % r.getpeername()[1], data.decode('utf-8'))
                            if self.users[r]:  # 将已注册用户的信息广播给其他已注册用户
                                # TODO
                                for u in self.users:
                                    if u not in self.wlist and u is not r and u is not self.server:
                                        self.wlist.append(u)
                                        self.message_queues[u] = queue.Queue()
                                    self.message_queues[u].put(data)
                            else:  # 对未注册用户发送的昵称进行检验，并进行注册
                                # TODO
                                if r not in self.wlist:
                                    self.wlist.append(r)
                                if data.decode('utf-8') in self.users.values():
                                    self.message_queues[r].put('Username already exists!'.encode('utf-8'))
                                else:
                                    self.message_queues[r].put('Welcome!'.encode('utf-8'))
                                    self.users[r] = data.decode('utf-8')
                        else:
                            print('客户端断开了')
                            if r in self.wlist:
                                self.wlist.remove(r)   # 清理已断开的连接
                            self.rlist.remove(r)        # 清理已断开的连接
                            del self.message_queues[r]  # 清理已断开的连接
                    except ConnectionResetError:
                        print("客户端异常断开了", r)
                        if r in self.wlist:
                            self.wlist.remove(r)   # 清理已断开的连接
                        self.rlist.remove(r)        # 清理已断开的连接
                        del self.message_queues[r]  # 清理已断开的连接
            for w in ws:       # 遍历可写的 fd 列表，即准备好发送数据的那些fd
                # 判断队列是否为空
                try :
                    next_msg = self.message_queues[w].get_nowait()
                except queue.Empty:
                    # print("client [%s]" % w.getpeername()[0], "queue is empty..")
                    self.wlist.remove(w)
                # 队列不为空，就把队列中的数据改成大写，原样发回去
                else:
                    print("发送数据到 [%s]"% w.getpeername()[0], next_msg)
                    w.send(next_msg)

            for e in xs:   # 处理报错的 fd
                e.close()
                print("Error occured in ",e.getpeername())
                self.rlist.remove(e)
                if e in self.wlist:
                    self.wlist.remove(e)
                del self.message_queues[e] 

if __name__=='__main__':
    host = '0.0.0.0'      # 主机名
    port = 9999           # 端口号
    server = Server(host,port)
    server.accept()

new connection from ('127.0.0.1', 14095)
new connection from ('127.0.0.1', 14096)
收到来自[14095]的数据: Zhang San
收到来自[14096]的数据: Zhang San
发送数据到 [127.0.0.1] b'Welcome!'
发送数据到 [127.0.0.1] b'Username already exists!'
收到来自[14096]的数据: Xiao Ming
发送数据到 [127.0.0.1] b'Welcome!'
收到来自[14095]的数据: Hello everybody!
发送数据到 [127.0.0.1] b'Hello everybody!'
客户端断开了
客户端断开了


KeyboardInterrupt: 

## 第二部分 Pandas基础练习（4.5分）

### 2.1 基本操作

In [285]:
import pandas as pd
import numpy as np

data = {
    'age': [24, 53, 23, 24, 33, 42, 57, 57, 36, 29, 53, 39],
    'gender': ['M', 'F', 'M', 'M', 'F', 'M', 'M', 'M', 'M', 'M', 'M', 'F'],
    'occupation': ['technician', 'other', 'writer', 'technician', 'technician', 'executive', 'administrator', 'administrator', 'administrator', 'student', 'lawyer', 'other'],
    'zip_code': ['85711', '94043', '32067', '43537', '15213', '98101', '91344', '91344', '05201', '01002', '90703', '30329']
}

创建一组从1开始递增的id，作为行索引，由data生成一个DataFrame。并以gender分组，打印每组的前两条。（0.5分）

In [286]:
# TODO
df = pd.DataFrame(data, index=list(range(1, len(data['age'])+1)))
df_gender_group = df.groupby('gender')
df_gender_group.apply(lambda x: x.head(2))

age gender  occupation zip_code
gender                                   
F      2   53      F       other    94043
       5   33      F  technician    15213
M      1   24      M  technician    85711
       3   23      M      writer    32067

删除DataFrame中重复的行（保留一行），再删除所有occupation为"other"的行。（0.5分）

In [287]:
# TODO
df = df.drop_duplicates()
df = df.drop(df.loc[df['occupation'] == 'other'].index)
print(df)

    age gender     occupation zip_code
1    24      M     technician    85711
3    23      M         writer    32067
4    24      M     technician    43537
5    33      F     technician    15213
6    42      M      executive    98101
7    57      M  administrator    91344
9    36      M  administrator    05201
10   29      M        student    01002
11   53      M         lawyer    90703


以两种方式访问DataFrame的第一行。（0.5分）

In [288]:
# TODO
print(df[:1])
print('-------------------------------------')
print(df.iloc[:1])

   age gender  occupation zip_code
1   24      M  technician    85711
-------------------------------------
   age gender  occupation zip_code
1   24      M  technician    85711


以两种方式访问DataFrame的"occupation"列。（0.5分）

In [289]:
# TODO
print(df['occupation'])
print('-------------------------------------')
print(df.occupation)

1        technician
3            writer
4        technician
5        technician
6         executive
7     administrator
9     administrator
10          student
11           lawyer
Name: occupation, dtype: object
-------------------------------------
1        technician
3            writer
4        technician
5        technician
6         executive
7     administrator
9     administrator
10          student
11           lawyer
Name: occupation, dtype: object


打印出DataFrame中出现次数最多的职业名称（0.5分）

In [290]:
# TODO
print(df['occupation'].value_counts().head(1).index[0])

technician


将data_2转换成DataFrame，与上一个DataFrame按照"zip_code"这一列合并，按两种合并方式：（0.5分）

1.合并后结果保留两个DataFrame的并集；

2.合并后只保留data_2的所有数据。

In [291]:
data_2 = {
    'zip_code': ['85711', '94043', '15213', '98101', '30329'],
    'name': ['Alan', 'Alice', 'Abigail', 'Bob', 'Dave']
}
df2 = pd.DataFrame(data_2)

# TODO
df_union_1 = pd.merge(df, df2, how='outer')
print(df_union_1)
print('----------------------------------------')
df_union_2 = pd.merge(df, df2, how='right')
print(df_union_2)

     age gender     occupation zip_code     name
0   24.0      M     technician    85711     Alan
1   23.0      M         writer    32067      NaN
2   24.0      M     technician    43537      NaN
3   33.0      F     technician    15213  Abigail
4   42.0      M      executive    98101      Bob
5   57.0      M  administrator    91344      NaN
6   36.0      M  administrator    05201      NaN
7   29.0      M        student    01002      NaN
8   53.0      M         lawyer    90703      NaN
9    NaN    NaN            NaN    94043    Alice
10   NaN    NaN            NaN    30329     Dave
----------------------------------------
    age gender  occupation zip_code     name
0  24.0      M  technician    85711     Alan
1   NaN    NaN         NaN    94043    Alice
2  33.0      F  technician    15213  Abigail
3  42.0      M   executive    98101      Bob
4   NaN    NaN         NaN    30329     Dave


### 2.2 多级索引

In [292]:
# 构建多级索引DataFrame
# int数据由 np.random.randint(0, 150, size=(6, 3) 产生
# from tuples, from_arrays, from_product
import numpy as np
import pandas as pd

df3 = pd.DataFrame(np.random.randint(0, 150, size=(6, 3)),
                  columns=["Python", "C++", "Java"],
                  index=pd.MultiIndex.from_product([['张三', '李四', '王五'],
                                                  ['期中', '期末']]))
df3

Python  C++  Java
张三 期中       3   21    33
   期末     108   43    94
李四 期中     100    2     3
   期末      95  140    24
王五 期中      82   35   128
   期末      91  101   135

实现变换df3为三重索引的serial。(0.5分)

In [293]:
# TODO
df3_serial = pd.Series(df3.values.reshape(-1), 
    index=pd.MultiIndex.from_product([*df3.index.levels, list(df3.columns)])
        )
print(df3_serial)

张三  期中  Python      3
        C++        21
        Java       33
    期末  Python    108
        C++        43
        Java       94
李四  期中  Python    100
        C++         2
        Java        3
    期末  Python     95
        C++       140
        Java       24
王五  期中  Python     82
        C++        35
        Java      128
    期末  Python     91
        C++       101
        Java      135
dtype: int32


按期中40%，期末60%的比例，计算输出每位同学每门功课的总评成绩表。（0.5分）

In [294]:
# TODO
df3_final = df3.copy()
df3_final.loc[(slice(None), '期中'), :] *= 0.4
df3_final.loc[(slice(None), '期末'), :] *= 0.6
df3_final = df3_final.sum(axis=0, level=0)
df3_final

e:\Anaconda3\envs\gomoku\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  """


,Python,C++,Java
张三,66.0,34.2,69.6
李四,97.0,84.8,15.6
王五,87.4,74.6,132.2


计算输出每门功课的最高分，最低分，以及平均分数表（课程名做行索引，列名用最高分，最低分，平均分数）。（0.5分）

In [295]:
# TODO
df3_final_2 = df3_final.T
df3_final_2_max = df3_final_2.max(axis=1)
df3_final_2_min = df3_final_2.min(axis=1)
df3_final_2_avg = df3_final_2.mean(axis=1)
df3_final_2 = pd.concat([df3_final_2_max, df3_final_2_min, df3_final_2_avg], axis=1)
df3_final_2.columns = ['最高分', '最低分', '平均分数']
df3_final_2

,最高分,最低分,平均分数
Python,97.0,66.0,83.466667
C++,84.8,34.2,64.533333
Java,132.2,15.6,72.466667


## 第三部分 Pandas数据分析（3分）

使用Pandas的read_csv函数读取 IBM_2006-01-01_to_2018-01-01.csv 中的数据，得到一个DataFrame，通过shape和info()查看数据基本信息，并打印该数据的前10行观察一下。该数据列索引对应的分别是：日期，开盘价，最高价，最低价，收盘价，成交量，股票名称。（0.5分）

In [314]:
import pandas as pd

IBM_file = 'IBM_2006-01-01_to_2018-01-01.csv'
# TODO
df = pd.read_csv(IBM_file)
print(df.shape)
print(df.info())
print(df.head(10))

(3020, 7)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3020 entries, 0 to 3019
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Date    3020 non-null   object 
 1   Open    3019 non-null   float64
 2   High    3020 non-null   float64
 3   Low     3019 non-null   float64
 4   Close   3020 non-null   float64
 5   Volume  3020 non-null   int64  
 6   Name    3020 non-null   object 
dtypes: float64(4), int64(1), object(2)
memory usage: 165.3+ KB
None
         Date   Open   High    Low  Close    Volume Name
0  2006-01-03  82.45  82.55  80.81  82.06  11715200  IBM
1  2006-01-04  82.20  82.50  81.33  81.95   9840600  IBM
2  2006-01-05  81.40  82.90  81.00  82.50   7213500  IBM
3  2006-01-06  83.95  85.03  83.41  84.95   8197400  IBM
4  2006-01-09  84.10  84.25  83.38  83.73   6858200  IBM
5  2006-01-10  83.15  84.12  83.12  84.07   5701000  IBM
6  2006-01-11  84.37  84.81  83.40  84.17   5776500  IBM
7  2006-01-12  83.82  83.9

观察到Open与Low这两列相比其他列都缺了一项，请删除DataFrame中所有不完整的数据。（0.5分）

In [315]:
# TODO
df_drop_null = df.dropna(axis='index', how='any')
df_drop_null

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
1,2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2,2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
3,2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
4,2006-01-09,84.10,84.25,83.38,83.73,6858200,IBM
...,...,...,...,...,...,...,...
3015,2017-12-22,151.82,153.00,151.50,152.50,2990583,IBM
3016,2017-12-26,152.51,153.86,152.50,152.83,2479017,IBM
3017,2017-12-27,152.95,153.18,152.61,153.13,2149257,IBM
3018,2017-12-28,153.20,154.12,153.20,154.04,2687624,IBM


由于股市周末和节假日不开盘，日期为2006-01-06的下一条数据就是2006-01-09，缺少了中间两天的数据。请将所有缺少的数据按照以下规则补充：成交量（volume）填充为零，开盘价等价格与前一天（最近的有数据的一天）保持相同。（0.5分）

提示：使用pandas.date_range生成时间序列

In [358]:
# TODO
new_df = df.copy()
for i in range(len(df.index)-1):
    dts = pd.date_range(df['Date'][df.index[i]], df['Date'][df.index[i+1]])
    if len(dts) != 2:
        for dt in dts[1:-1]:
            new_df = pd.concat([new_df, df.iloc[i:i+1]], ignore_index=True)
            new_df.iloc[-1:, 0] = str(dt)[:-9]
            new_df.iloc[-1:, -2] = 0
new_df = new_df.sort_values('Date')
new_df

,Date,Open,High,Low,Close,Volume,Name
0,2006-01-03,82.45,82.55,80.81,82.06,11715200,IBM
1,2006-01-04,82.20,82.50,81.33,81.95,9840600,IBM
2,2006-01-05,81.40,82.90,81.00,82.50,7213500,IBM
3,2006-01-06,83.95,85.03,83.41,84.95,8197400,IBM
3020,2006-01-07,83.95,85.03,83.41,84.95,0,IBM
...,...,...,...,...,...,...,...
4378,2017-12-25,151.82,153.00,151.50,152.50,0,IBM
3016,2017-12-26,152.51,153.86,152.50,152.83,2479017,IBM
3017,2017-12-27,152.95,153.18,152.61,153.13,2149257,IBM
3018,2017-12-28,153.20,154.12,153.20,154.04,2687624,IBM


在DataFrame中按月份聚合，计算一年当中每个月的成交量总和，以及每个月收盘价的次高值。(1分)

提示：可以通过apply方法实现自定义的聚合函数。（有多个相同值的话，这里先简单地按照排完序取下标第二个就行了）

In [372]:
# TODO
df_pregroup = df.copy()
df_pregroup.insert(1, 'Month', None)
df_pregroup['Month'] = ((lambda x: [i[:7] for i in x])(df_pregroup['Date']))
df_grouped = df_pregroup.groupby('Month')
print('------成交量总和------')
d = df_grouped.apply(lambda x: x.sum()['Volume'])
print(d)
print('-----收盘价次高值-----')
d_2 = df_grouped.apply(lambda x: x.sort_values('Close', ascending=False).iloc[1, -3])
print(d_2)

------成交量总和------
Month
2006-01    146611300
2006-02    111061200
2006-03    111811000
2006-04    114030800
2006-05    113238100
             ...    
2017-08     80295410
2017-09     78281970
2017-10    121897687
2017-11     89776130
2017-12     88788538
Length: 144, dtype: int64
-----收盘价次高值-----
Month
2006-01     84.46
2006-02     81.35
2006-03     83.81
2006-04     83.88
2006-05     83.23
            ...  
2017-08    145.16
2017-09    145.99
2017-10    160.90
2017-11    153.97
2017-12    156.46
Length: 144, dtype: float64


读取AMZN_2006-01-01_to_2018-01-01.csv，其中包含了Amazon的股票数据。
请以Date作为索引，'IBM'和'AMZN'作为列索引，构建一个Dataframe，其中的数据为两家公司每天的股票收盘价。（0.5分）

In [392]:
AMZN_file = 'AMZN_2006-01-01_to_2018-01-01.csv'
IBM_file = 'IBM_2006-01-01_to_2018-01-01.csv'

AMZN_data = pd.read_csv(AMZN_file)
IBM_data = pd.read_csv(IBM_file)

# TODO
AMZN_data_reset = AMZN_data[['Date', 'Close']].rename(columns={'Close':'AMZN'})
IBM_data_reset = IBM_data[['Date', 'Close']].rename(columns={'Close':'IBM'})
data_union = pd.merge(AMZN_data_reset, IBM_data_reset, how='outer').set_index('Date')
data_union

,AMZN,IBM
Date,,
2006-01-03,47.58,82.06
2006-01-04,47.25,81.95
2006-01-05,47.65,82.50
2006-01-06,47.87,84.95
2006-01-09,47.08,83.73
...,...,...
2017-12-26,1176.76,152.83
2017-12-27,1182.26,153.13
2017-12-28,1186.10,154.04
